In [4]:
#!pip install jiwer

In [6]:
from jiwer import wer
import random
import pandas as pd

In [ ]:
reference = """
Boa tarde, senhor João Pereira. Gostaria de começar explicando que esta entrevista está sendo gravada. 
O senhor tem o direito de permanecer em silêncio e de contar com um advogado durante todo o processo. 
Entendeu seus direitos? Sim. Ótimo. Vamos conversar sobre um incidente ocorrido recentemente em Lisboa, 
envolvendo a senhora Joana Castro. Ela relatou que sua mala foi roubada por volta das 20:00, no Colombo.
Estamos aqui para esclarecer os fatos. Está disposto a colaborar?
""".replace("\n", " ").strip()

# Gerar transcrição com erro
def gerar_transcricao_errada(texto, erro_desejado=0.18):
    palavras = texto.split()
    n_total = len(palavras)
    n_erros = round(n_total * erro_desejado)

    for _ in range(n_erros):
        i = random.randint(0, len(palavras) - 1)
        acao = random.choice(["substituir", "omitir", "inserir"])

        if acao == "substituir":
            palavras[i] = "ERRO"
        elif acao == "omitir":
            palavras[i] = ""
        elif acao == "inserir":
            palavras.insert(i, "EXTRA")

    palavras = [p for p in palavras if p != ""]
    return " ".join(palavras)

transcricao = gerar_transcricao_errada(reference)

print("WER:", wer(reference, transcricao))



WER: 0.14473684210526316


In [11]:
def gerar_transcricao_errada(texto, erro_desejado=0.18):
    palavras = texto.split()
    n_total = len(palavras)
    n_erros = round(n_total * erro_desejado)

    for _ in range(n_erros):
        if not palavras:
            break
        i = random.randint(0, len(palavras) - 1)
        acao = random.choice(["substituir", "omitir", "inserir"])
        if acao == "substituir":
            palavras[i] = "ERRO"
        elif acao == "omitir":
            palavras[i] = ""
        elif acao == "inserir":
            palavras.insert(i, "EXTRA")
    palavras = [p for p in palavras if p != ""]
    return " ".join(palavras)
df = pd.read_csv("C://Users//garci//OneDrive//Área de Trabalho//Ambiente de trabalho unbunto//Cienciadedados//IPL//Tese//interrogatorio_respostas.csv")
df["transcricao_simulada"] = df["referencia"].apply(lambda x: gerar_transcricao_errada(str(x), erro_desejado=0.18))
df.to_csv("com_transcricao_simulada.csv", index=False)


In [14]:
df = pd.read_csv("C://Users//garci//OneDrive//Área de Trabalho//Ambiente de trabalho unbunto//Cienciadedados//IPL//Tese//com_transcricao_simulada.csv")
df["wer"] = df.apply(lambda row: wer(str(row["referencia"]), str(row["transcricao_simulada"])), axis=1)
print(df[["referencia", "transcricao_simulada", "wer"]].head())
media_wer = df["wer"].mean()
print(f"\nWER médio: {media_wer:.2%}")


                                          referencia  \
0  "Boa tarde, senhor João pereira. Gostaria de c...   
1                                             "Sim."   
2  "Ótimo. Vamos conversar sobre um incidente oco...   
3  "Antes de tudo, pode me contar onde o senhor e...   
4  Encontrava-me nas Compras no Continente do Col...   

                                transcricao_simulada       wer  
0  "Boa tarde, senhor João pereira. Gostaria de c...  0.135135  
1                                             "Sim."  0.000000  
2  "Ótimo. Vamos sobre um incidente ocorrido rece...  0.157895  
3  "Antes de tudo, pode me contar onde o senhor e...  0.142857  
4  Encontrava-me nas Compras no Continente do Col...  0.090909  

WER médio: 15.93%
